This model predicts the makeup of the travel party (adults, children, or mixed).

In [1]:
import os
import toml
import pandas as pd
# import validation_data_input
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import psrc_theme

import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected" # to show plotly figures in quarto HTML file
pio.templates.default = "simple_white+psrc_color" # set plotly template


config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

# run get_data.ipynb to load validation data
%store -r validation_data

In [2]:
tour_data = validation_data.tours.copy()
# all joint tours
joint_tour_data = tour_data.loc[tour_data['tour_category'] == "joint"]

- joint tour count

In [3]:
# total number of joint tours by source
df_1 = joint_tour_data.groupby('source')['tour_weight'].sum().reset_index()
df_1 = df_1.rename(columns={"tour_weight": "joint tour count"})
# total number of tours by source
df_2 = tour_data.groupby('source')['tour_weight'].sum().reset_index()
df_2 = df_2.rename(columns={"tour_weight": "tour count"})

df_joint_tour_count = df_1.merge(df_2, how="left", on='source')
df_joint_tour_count['joint_percent'] = df_joint_tour_count['joint tour count']/df_joint_tour_count['tour count']
df_joint_tour_count['tour count'] = df_joint_tour_count['tour count'].astype(float).map(lambda n: '{:.2f}'.format(n))
df_joint_tour_count['joint tour count'] = df_joint_tour_count['joint tour count'].astype(float).map(lambda n: '{:.2f}'.format(n))
df_joint_tour_count['joint_percent'] = df_joint_tour_count['joint_percent'].astype(float).map(lambda n: '{:.2%}'.format(n))
df_joint_tour_count

,source,joint tour count,tour count,joint_percent
0,model results,114114.00,4946345.00,2.31%
1,survey data,525347.70,4633302.91,11.34%
2,unweighted survey,1014.00,11476.00,8.84%


In [4]:
df_plot = joint_tour_data.groupby(['source','composition'])[['tour_weight']].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['tour_weight']. \
    apply(lambda x: x / float(x.sum()))
# df_plot
fig = px.bar(df_plot, x="composition", y="percentage", color="source",barmode="group", 
             # color_discrete_sequence=config['psrc_color'],
             title="joint tour composition")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, font=dict(size=11), yaxis=dict(tickformat=".2%"), xaxis=dict(dtick="1"))
fig.show()